<a href="https://colab.research.google.com/github/yerimmms/calibrate-moe/blob/calibrate-moe/codebench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/CodeEditorBench/CodeEditorBench.git

Cloning into 'CodeEditorBench'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 451 (delta 4), reused 1 (delta 1), pack-reused 443
Receiving objects: 100% (451/451), 3.66 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [97]:
# colab 용
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
%cd CodeEditorBench/
!ls
!conda env create -f CodeEditorBench/coder.yml

In [98]:
!conda env list

# conda environments:
#
base                     /usr/local
coder                    /usr/local/envs/coder



In [35]:
!conda activate coder

In [ ]:
!huggingface-cli login

In [ ]:
# %cd data
filenames = [
    "code_debug_plus.jsonl",
    "code_debug_primary.jsonl",
    "code_polishment_plus.jsonl",
    "code_polishment_primary.jsonl",
    "code_switch_plus.jsonl",
    "code_switch_primary.jsonl",
    "code_translate_plus.jsonl",
    "code_translate_primary.jsonl",
]

# !huggingface-cli download m-a-p/CodeEditorBench {filenames[0]} --repo-type=dataset --local-dir="." --local-dir-use-symlinks="auto"
# download filenames with huggingface-cli
!mkdir data
for filename in filenames:
    !huggingface-cli download m-a-p/CodeEditorBench {filename} --repo-type=dataset --local-dir="./data/" --local-dir-use-symlinks="auto"

In [ ]:
!huggingface-cli download --resume-download codellama/CodeLlama-7b-Instruct-hf --local-dir ./CodeLlama-7b-Instruct-hf

In [65]:
!mkdir -p greedy_result/{code_debug,code_translate,code_polishment,code_switch}
!ls -l

total 108
-rw-r--r-- 1 root root  2540 Jun  4 01:49 api_handler.py
-rw-r--r-- 1 root root 15153 Jun  4 01:49 api_inference.py
drwxr-xr-x 2 root root  4096 Jun  4 01:49 cluster
-rw-r--r-- 1 root root  2894 Jun  4 01:49 coder.yml
drwxr-xr-x 3 root root  4096 Jun  4 02:21 data
-rw-r--r-- 1 root root   991 Jun  4 01:49 dataset.py
drwxr-xr-x 3 root root  4096 Jun  4 01:49 evaluation
drwxr-xr-x 4 root root  4096 Jun  4 01:49 few_shot_prompt
drwxr-xr-x 6 root root  4096 Jun  4 02:37 greedy_result
-rw-r--r-- 1 root root 11431 Jun  4 01:49 LICENSE
drwxr-xr-x 2 root root  4096 Jun  4 01:49 mdPICs
drwxr-xr-x 4 root root  4096 Jun  4 02:37 model
drwxr-xr-x 2 root root  4096 Jun  4 01:49 prompt_function
-rw-r--r-- 1 root root  7251 Jun  4 01:49 README.md
-rw-r--r-- 1 root root 11231 Jun  4 01:49 result_postprocess.py
-rw-r--r-- 1 root root  9516 Jun  4 01:49 vllm_inference.py
-rw-r--r-- 1 root root  1154 Jun  4 01:49 vllm_inference.sh


In [ ]:
!python vllm_inference.py \
    --base_model "CodeLlama-7b-Instruct-hf" \
    --dataset "code_debug" \
    --input_data_dir "./data/" \
    --output_data_dir "./greedy_result/" \
    --batch_size 64 \
    --num_of_sequences 1 \
    --num_gpus 1 \
    --prompt_type "zero" \
    --start_idx 0 \
    --end_idx -1